# Setup CC359 Dataset

In [ ]:
import glob
import shutil
import zipfile
from pathlib import Path

path = Path("/tmp/data/CC359")
dl_path = path / "download"

vendors = ["philips_3", "philips_15", "siemens_3", "siemens_15", "ge_15", "ge_3"]

In [ ]:
# ----- Extract the gdrive zip file -----

with zipfile.ZipFile(path / "CC359.zip", "r") as zip_file:
    zip_file.extractall(path)

# rename
shutil.move(path / "CC359", path / dl_path)
# os.remove(path / 'CC359.zip')

In [ ]:
# ----- Extract Original.zip and re-organize -----

with zipfile.ZipFile(dl_path / "Original.zip", "r") as zip_file:
    zip_file.extractall(path)

# os.remove(dl_path / 'Original.zip')

for vendor in vendors:
    vendor_path = path / "Original" / vendor.upper()
    vendor_path.mkdir(parents=True)

    for file_path in glob.iglob(str(path / "Original" / f"CC*_{vendor}_*.nii.gz")):
        shutil.move(file_path, vendor_path)

In [ ]:
# ----- Extract Silver-standard-STAPLE.zip and re-organize -----
silver_standard = "Silver-standard-machine-learning"

with zipfile.ZipFile(dl_path / "Skull-stripping-masks" / f"{silver_standard}.zip", "r") as zip_file:
    zip_file.extractall(path)

# rename
# os.remove(dl_path / 'Skull-stripping-masks' / f'{silver_standard}.zip')

for vendor in vendors:
    vendor_path = path / "Silver-standard" / vendor.upper()
    vendor_path.mkdir(parents=True)

    for file_path in glob.iglob(str(path / "Silver-standard" / f"CC*_{vendor}_*.nii.gz")):
        shutil.move(file_path, vendor_path)

In [ ]:
# final cleanup
shutil.rmtree(path / "__MACOSX")

## Upload to W&B

In [1]:
import wandb
from pathlib import Path

data_dir = Path("/tmp/data/CC359")

In [ ]:
run = wandb.init(project="UDA-Datasets")

artifact = wandb.Artifact("CC359-Skull-stripping", type="raw_data")
artifact.add_dir(data_dir)
run.log_artifact(artifact)

wandb.finish()

In [3]:
api = wandb.Api()
run = api.run(f"iserh/UDA-Datasets/{run.id}")

run.delete()